In [2]:
import os
os.environ["JUPYTER_COLUMNS"] = "80"
# %load_ext rich

## About myself {.smaller}

:::: {.columns}
::: {.column width="40%"}
:::: {.content-visible when-format="html"}
![](./img/memes/myself.gif){height="12em"}
::::
:::

::: {.column width="60%"}

::: {.incremental}
- 🐍 Pythonista with 18 years of experience 🇦🇷 🧉
  - Co-organized SciPy Latin America
  - PyCon Argentina and EuroPython speaker
- 🚜 Worked as CTO in  Hello Tractor 
- 🧪 Software Engineer at IBM Research 
- 🛰️ Worked in [Foundational Models for Geospatial applications {{< bi link >}}](https://www.earthdata.nasa.gov/news/nasa-ibm-openly-release-geospatial-ai-foundation-model-nasa-earth-observation-data)
- 💬 Currently working on [Flowpilot {{< bi link >}}](https://research.ibm.com/projects/flowpilot), providing core features to 
  different products and divisions
::: 

:::
::::

---

::: {.content-visible when-format="html"}

## Tomorrow {.center}

::: {.columns}
::: {.column width="30%"}
![](./img/promo/wanjiru.jpg)
:::
::: {.column width="30%"}
![](./img/promo/beldine.jpg)
:::
::: {.column width="30%"}
![](./img/promo/reggie.png)
:::
::: 

Learn about [Multimodal Geospatial Foundation Models with Terratorch](https://pycon-kenya-2025.sessionize.com/session/954933){target="_blank"}
 by Wanjiru, Beldine and Reggie

:::

---

::: {.content-visible when-format="html"}

## Follow along (or at 🏡) {.center}

::: {.columns}
::: {.column}
::: {style="text-align: center;"}
<a href="https://d3f0.github.io/pycon-sql-llm/slides.html#/title-slide" target="_blank">

{{< qrcode https://d3f0.github.io/pycon-sql-llm/slides.html#/title-slide qr1 width=400 height=400 colorDark='#0011bb' >}}

</a>

Get the slides {{< bi arrow-up-circle >}}
:::

:::
::: {.column}
::: {style="font-size: .67em"}
Get the code {{< bi arrow-down-circle >}}

[{{< bi github >}} https://github.com/D3f0/pycon-sql-llm](https://github.com/D3f0/pycon-sql-llm)
:::
:::
:::

---

# Agenda

In this talk we're gonna show how to use Python to:

  - {{< bi database >}} Connect to a database and execute the queries
  - {{< bi filetype-sql >}} Convert natural language questions into SQL
  - {{< bi diagram-3 >}} Create a workflow
  - {{< bi database-gear >}} Managing configuration
  - {{< bi bookmark-check >}} Lessons learned

---

## Pipeline Overview

![](./img/LLMsSQL.svg)



---

# DB Inspection
<!-- ## Connect to a database and execute the queries -->

---

### Public datasets used in text to SQL

:::: {.columns}

::: {.column}
::: {.fragment .fade-left .small_text}
These datasets define:

- ❓ Natural language questions
- 🤞🏽 Expected SQL
- 🏗️ Database schema & content
- 🔎 Evaluation metrics 
- 🥇 Leaderboard
:::
:::

::: {.column}

::: {.fragment .fade-right}
- [🕷️ Spider](https://yale-lily.github.io/spider){target="_bank"}
:::
::: {.fragment .fade-right}
- [🕷️ 🕷️ Spider 2](https://spider2-sql.github.io/){target="_bank"}
:::
::: {.fragment .fade-right}
- 🐦 [BIRD](https://bird-bench.github.io/){target="_bank"}
:::
::: {.fragment .fade-right}
- [🏹 Archer](https://sig4kg.github.io/archer-bench/){target="_bank"}
:::
:::

::::


::: notes
There are various academic datasets that are used for advancing the field
of text2sql.
:::

---

## BIRD 

![[IBM Research in the leaderboard last year](https://research.ibm.com/blog/granite-LLM-text-to-SQL)](./img/ibm-granite-leaderboard.png){height="400px" fig-align="center"}


::: notes
We're select BIRD dataset since we have some experience with it, we managed
to get to the top of the leatherboard in 2024/6
:::

---

### BIRD mini-dev

- It consist of 500 queries classified as **simple**, **moderate** and **challenging** [{{< bi box-arrow-up-right >}}](https://github.com/bird-bench/mini_dev)

::: {.fragment .fade-left}

<!-- Setup cell without output with imports -->

In [3]:
from rich.console import Console
_c = Console(width=45)
# Ensure print are contained within the range
print = _c.print


from IPython.display import display, Image

In [4]:
#| echo: true
from datasets import load_dataset, DownloadConfig 

# Load the dataset
dataset = load_dataset(
  "birdsql/bird_mini_dev", 
  download_config=DownloadConfig(disable_tqdm=True)
)

# Only SQLite (there's Postgres and MySQL in the same dataset)
sqlite_df = (dataset["mini_dev_sqlite"]
  .to_pandas()
  .pipe(lambda df: df.drop(columns=['question_id'])))
display(sqlite_df.sample(5, random_state=17))

,db_id,question,evidence,SQL,difficulty
190,formula_1,For the constructor which got the highest poin...,race number refers to raceId; constructor whic...,SELECT T2.url FROM constructorResults AS T1 IN...,moderate
267,superhero,"In superheroes with height between 150 to 180,...",height between 150 to 180 refers to height_cm ...,SELECT CAST(COUNT(CASE WHEN T2.publisher_name ...,challenging
457,california_schools,What is the monthly average number of schools ...,Elementary School District refers to DOC = 52;...,SELECT CAST(COUNT(School) AS REAL) / 12 FROM s...,moderate
263,superhero,What is the average weight of all female super...,female refers to gender = 'Female'; average we...,SELECT AVG(T1.weight_kg) FROM superhero AS T1 ...,simple
159,european_football_2,When was the first time did Kevin Constant hav...,Kevin Constant refers to player_name = 'Kevin ...,"SELECT `date` FROM ( SELECT t2.crossing, t2.`d...",moderate


:::

---

### Downloading BIRD databases

<!-- - We will be using [`uvx`](https://docs.astral.sh/uv/concepts/tools/#tool-versions)[^uvx] with the `gdown` package as follows: -->

```shell
uvx gdown https://drive.google.com/file/d/13VLWIwpw5E3d5DUkMvzw7hvHE67a4XkG/
```
![](./img/download_minidev.png)
<!-- Or just download from the [ {{< bi google >}} {{< bi hdd >}} link](https://drive.google.com/file/d/13VLWIwpw5E3d5DUkMvzw7hvHE67a4XkG/view?usp=sharing)  -->

Extracting the archive (3.3GiB)

```shell
unzip minidev_703.zip
```
<!-- [^uvx]: It comes as part of `uv`, it's a shorthand for `uv tool run` -->

::: footer
The dataset doesn't contain the SQL flies, these are shared separately as zipfiles. 
In this case it's located in Google Drive.
:::
---

### Picking the example database `california_schools`

In `minidev/MINIDEV/dev_databases/california_schools/` we find the {{< bi database >}} {{< bi file >}}

![](./img/california_school.png)

In [5]:
#| echo: true
#| code-overflow: wrap
from pathlib import Path
base = Path('./minidev/MINIDEV/dev_databases/california_schools/')

---

## ER of `california_schools` {{< bi database >}}

In [6]:
#from eralchemy import render_er
#render_er(db_url, './img/output/erd_from_sqlite.svg')

![](./img/er-california-schools.png){.lightbox}

::: footer
:::

---


### Creating the `Engine`

Creating an `Engine` instance connected to a SQLite database to run our queries.

In [7]:
#| echo: true
#| code-overflow: wrap

from sqlalchemy import create_engine
db_path = base / 'california_schools.sqlite'
db_url = f"sqlite:///{db_path}"
engine = create_engine(db_url)
engine

Engine(sqlite:///minidev/MINIDEV/dev_databases/california_schools/california_schools.sqlite)

::: notes
This allows us to execute, manage transactions and do database introspection.
:::


---

### Getting one simple question 

In [8]:
#| echo: true
simple_queries_df = (sqlite_df
  .pipe(lambda df: df[df.db_id == 'california_schools'])
  .pipe(lambda df: df[df.difficulty == 'simple']))

simple_queries_df.head(2).set_index("db_id")

,question,evidence,SQL,difficulty
db_id,,,,
california_schools,How many schools with an average score in Math...,Exclusively virtual refers to Virtual = 'F',SELECT COUNT(DISTINCT T2.School) FROM satscore...,simple
california_schools,Please list the codes of the schools with a to...,Total enrollment can be represented by `Enroll...,SELECT T2.CDSCode FROM schools AS T1 INNER JOI...,simple


---

### Getting a `simple` question and query

In [9]:
#| echo: false
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [10]:
#| echo: true


question_sql_df = (simple_queries_df
  .pipe(lambda df: df[["question", "SQL"]])
  .reset_index())

::: {.fragment .fade-left}
Let's take a look at the `question` and `SQL` column

In [11]:
#| echo: true

question, query = question_sql_df.loc[
  0, 
  ["question", "SQL"]]
display(question, query)

'How many schools with an average score in Math greater than 400 in the SAT test are exclusively virtual?'

"SELECT COUNT(DISTINCT T2.School) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T2.Virtual = 'F' AND T1.AvgScrMath > 400"

:::

::: notes
This is one of the valid SQL queries for this question
There may be other. When we're comparing performance, 
in situations where we have multiple rows and there
are no aggregations we need to decide wether the order is important or not
:::

---

### Execute the query {.smaller}

::: {style="font-size: 60%;"}
Now we run the `SQL` column captured in the variable `query` through SQLAlchemy and
plot the results as a DataFrame
:::

In [12]:
#| echo: true

from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text(query))
    res_df = pd.DataFrame(result.fetchall()) # 🐼 ✨
    display(res_df)

,COUNT(DISTINCT T2.School)
0,4


::: footer
:::


---

# Convert natural language questions into SQL {.smaller}

---

![](./img/LLMsSQL.svg)

---

## Coding models

:::: {.columns}
::: {.column}
::: {.fragment .fade-left}
LLMs are quite capable of writing functional SQL queries, from the named `code`/`coder`
to specific ones for (`sql`) generation.
:::

:::

::: {.column}

::: {.fragment .fade-left}
For example, some IBM trained models include:

- granite-3-2-8b-instruct
- granite-34b-code-instruct
- granite-20b-code-base-schema-linking
- granite-20b-code-base-sql-gen

[More info on these models {{< bi box-arrow-up-right >}}](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-ibm.html?context=wx&locale=en&audience=wdp#granite-code-instruct-models) | 
[And also in 🤗 {{< bi box-arrow-up-right >}}](https://huggingface.co/models?search=sql)

:::

:::

::::

::: notes
For those of us who have been writing code for some time and 
fell in love with ORMs when they were the *hot* new thing, LLMs
can take us to the next level!
:::

---

## Prompts for SQL generation {transition="convex"}


::: {.fragment .fade-in-then-semi-out}
LLMs don't know the  🏗️ structure of our database, and may hallucinate 
about it, or create some flat out invalid SQL
:::

::: {.fragment .fade-in-then-semi-out}
We have to  provide __extra__ information about the structure in the instructions

For this we will use a **prompt** string with some placeholders {{< bi braces >}}.
:::

::: {.fragment .fade-in-then-semi-out .small}
Some research papers from our team on Robust LLMs:

[{{< bi filetype-pdf >}} Weakly Supervised Detection of Hallucinations in LLMs](https://arxiv.org/pdf/2312.02798)
:::

::: {.fragment .fade-in-then-semi-out .small}
[{{< bi filetype-pdf >}} Localizing Persona Representations In LLMs](https://arxiv.org/pdf/2505.24539)
:::

::: notes
Now we're gonna see what we're put in the prompt, for starters the schema
:::

---


### Getting the schema for the prompt

LangChain (🦜 ⛓️) community 🐍 📦 provides a simple class that can retrieve some schema information
[SQL Question Answering](https://python.langchain.com/docs/tutorials/sql_qa/#system-prompt){target="_blank"}

::: {.small}
It implements the best practices as specified in: [{{< bi filetype-pdf >}} Rajkumar et al, 2022](https://arxiv.org/abs/2204.00498)
:::

In [13]:
#| echo: true
!uv pip install langchain-community

Audited 1 package in 38ms


In [14]:
#| echo: true

from langchain_community.utilities import SQLDatabase
db = SQLDatabase(engine=engine)

display(db.get_usable_table_names())

['frpm', 'satscores', 'schools']

::: notes
As we can see, the table names may not be immediately understandable 🤔
:::



---

###

In [15]:
#| echo: true
#| code-overflow: wrap
#| 
print(db.get_table_info())


CREATE TABLE frpm (
        "CDSCode" TEXT NOT NULL, 
        "Academic Year" TEXT, 
        "County Code" TEXT, 
        "District Code" INTEGER, 
        "School Code" TEXT, 
        "County Name" TEXT, 
        "District Name" TEXT, 
        "School Name" TEXT, 
        "District Type" TEXT, 
        "School Type" TEXT, 
        "Educational Option Type" TEXT, 
        "NSLP Provision Status" TEXT, 
        "Charter School (Y/N)" INTEGER, 
        "Charter School Number" TEXT, 
        "Charter Funding Type" TEXT, 
        "IRC" INTEGER, 
        "Low Grade" TEXT, 
        "High Grade" TEXT, 
        "Enrollment (K-12)" REAL, 
        "Free Meal Count (K-12)" REAL, 
        "Percent (%) Eligible Free (K-12)" 
REAL, 
        "FRPM Count (K-12)" REAL, 
        "Percent (%) Eligible FRPM (K-12)" 
REAL, 
        "Enrollment (Ages 5-17)" REAL, 
        "Free Meal Count (Ages 5-17)" REAL, 
        "Percent (%) Eligible Free (Ages 
5-17)" REAL, 
        "FRPM Count (Ages 5-17)" REAL, 
        "Percent (%) Eligible FRPM (Ages 
5-17)" REAL, 
        "2013-14 CALPADS Fall 1 Certification
Status" INTEGER, 
        PRIMARY KEY ("CDSCode"), 
        FOREIGN KEY("CDSCode") REFERENCES 
schools ("CDSCode")
)

/*
3 rows from frpm table:
CDSCode Academic Year   County Code     
District Code   School Code     County Name  
District Name   School Name     District Type
School Type     Educational Option Type NSLP 
Provision Status   Charter School (Y/N)    
Charter School Number   Charter Funding Type 
IRC     Low Grade       High Grade      
Enrollment (K-12)       Free Meal Count 
(K-12)  Percent (%) Eligible Free (K-12)     
FRPM Count (K-12)       Percent (%) Eligible 
FRPM (K-12)        Enrollment (Ages 5-17)  
Free Meal Count (Ages 5-17)     Percent (%) 
Eligible Free (Ages 5-17)   FRPM Count (Ages 
5-17)  Percent (%) Eligible FRPM (Ages 5-17) 
2013-14 CALPADS Fall 1 Certification Status
01100170109835  2014-2015       01      10017
0109835 Alameda Alameda County Office of 
Education      FAME Public Charter     County
Office of Education (COE)        K-12 Schools
(Public)   Traditional     None    1       
0728    Directly funded 1       K       12   
1087.0  565.0   0.519779208831647       715.0
0.657773689052438       1070.0  553.0   
0.516822429906542       702.0   
0.65607476635514        1
01100170112607  2014-2015       01      10017
0112607 Alameda Alameda County Office of 
Education      Envision Academy for Arts & 
Technology  County Office of Education (COE) 
High Schools (Public)   Traditional     None 
1       0811    Directly funded 1       9    
12      395.0   186.0   0.470886075949367    
186.0   0.470886075949367       376.0   182.0
0.484042553191489       182.0   
0.484042553191489       1
01100170118489  2014-2015       01      10017
0118489 Alameda Alameda County Office of 
Education      Aspire California College 
Preparatory Academy   County Office of 
Education (COE)        High Schools (Public) 
Traditional     None    1       1049    
Directly funded 1       9       12      244.0
134.0   0.549180327868853       175.0   
0.717213114754098       230.0   128.0   
0.556521739130435       168.0   
0.730434782608696       1
*/


CREATE TABLE satscores (
        cds TEXT NOT NULL, 
        rtype TEXT NOT NULL, 
        sname TEXT, 
        dname TEXT, 
        cname TEXT, 
        enroll12 INTEGER NOT NULL, 
        "NumTstTakr" INTEGER NOT NULL, 
        "AvgScrRead" INTEGER, 
        "AvgScrMath" INTEGER, 
        "AvgScrWrite" INTEGER, 
        "NumGE1500" INTEGER, 
        PRIMARY KEY (cds), 
        FOREIGN KEY(cds) REFERENCES schools 
("CDSCode")
)

/*
3 rows from satscores table:
cds     rtype   sname   dname   cname   
enroll12        NumTstTakr      AvgScrRead   
AvgScrMath      AvgScrWrite     NumGE1500
01100170000000  D       None    Alameda 
County Office of Education      Alameda 398  
88      418     418     417     14
01100170109835  S       FAME Public Charter  
Alameda County Office of Education      

<!-- Disable footer -->
::: footer
:::


---

### Prompts {.smaller}

In [16]:
#| echo: true
#| code-line-numbers: 1-7|8-14|15-18

system_message = """
Given an input question, create a syntactically correct {dialect} query to
run to help find the answer. Unless the user specifies in his question a
specific number of examples they wish to obtain, always limit your query to
at most {top_k} results. You can order the results by a relevant column to
return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema
description. Be careful to not query for columns that do not exist. Also,
pay attention to which column is in which table.

Only use the following tables:
{table_info}
"""

::: {.small_text}

[source {{< bi box-arrow-up-right >}}](https://python.langchain.com/docs/tutorials/sql_qa/#convert-question-to-sql-query)

:::

--- 

## The messages that constitute the prompt {.smaller}

Now we construct a list of messages. These are `dicts` which have a key
`user` or `system`, and a `content`.

In [17]:
#| echo: true
#| code-line-numbers: 4-10|11-14|19-23
def generate_messages(question, dialect="SQL", top_k=5, table_info=""):
    # Create a ChatPromptTemplate
    messages = [
      {
        "role": "system", 
        "content": system_message.format(
          dialect=dialect, 
          top_k=top_k, 
          table_info=table_info
        )},
      {
        "role": "user", 
        "content": question
      }
    ]
    
    return messages

messages = generate_messages(
  question=question, 
  dialect=db.dialect, top_k=10, 
  table_info=db.get_table_info())
messages

[{'role': 'system',
  'content': '\nGiven an input question, create a syntactically correct sqlite query to\nrun to help find the answer. Unless the user specifies in his question a\nspecific number of examples they wish to obtain, always limit your query to\nat most 10 results. You can order the results by a relevant column to\nreturn the most interesting examples in the database.\n\nNever query for all the columns from a specific table, only ask for a the\nfew relevant columns given the question.\n\nPay attention to use only the column names that you can see in the schema\ndescription. Be careful to not query for columns that do not exist. Also,\npay attention to which column is in which table.\n\nOnly use the following tables:\n\nCREATE TABLE frpm (\n\t"CDSCode" TEXT NOT NULL, \n\t"Academic Year" TEXT, \n\t"County Code" TEXT, \n\t"District Code" INTEGER, \n\t"School Code" TEXT, \n\t"County Name" TEXT, \n\t"District Name" TEXT, \n\t"School Name" TEXT, \n\t"District Type" TEXT, \n\t"S

In [18]:
#| echo: false
import json
with open("messages.json", "w") as fp:
    json.dump(messages, fp)

--- 

### 🔎 the `system` message

In [19]:
#| echo: false
#| classes: scrollable
print(messages[0]['content'])

Given an input question, create a 
syntactically correct sqlite query to
run to help find the answer. Unless the user 
specifies in his question a
specific number of examples they wish to 
obtain, always limit your query to
at most 10 results. You can order the results
by a relevant column to
return the most interesting examples in the 
database.

Never query for all the columns from a 
specific table, only ask for a the
few relevant columns given the question.

Pay attention to use only the column names 
that you can see in the schema
description. Be careful to not query for 
columns that do not exist. Also,
pay attention to which column is in which 
table.

Only use the following tables:

CREATE TABLE frpm (
        "CDSCode" TEXT NOT NULL, 
        "Academic Year" TEXT, 
        "County Code" TEXT, 
        "District Code" INTEGER, 
        "School Code" TEXT, 
        "County Name" TEXT, 
        "District Name" TEXT, 
        "School Name" TEXT, 
        "District Type" TEXT, 
        "School Type" TEXT, 
        "Educational Option Type" TEXT, 
        "NSLP Provision Status" TEXT, 
        "Charter School (Y/N)" INTEGER, 
        "Charter School Number" TEXT, 
        "Charter Funding Type" TEXT, 
        "IRC" INTEGER, 
        "Low Grade" TEXT, 
        "High Grade" TEXT, 
        "Enrollment (K-12)" REAL, 
        "Free Meal Count (K-12)" REAL, 
        "Percent (%) Eligible Free (K-12)" 
REAL, 
        "FRPM Count (K-12)" REAL, 
        "Percent (%) Eligible FRPM (K-12)" 
REAL, 
        "Enrollment (Ages 5-17)" REAL, 
        "Free Meal Count (Ages 5-17)" REAL, 
        "Percent (%) Eligible Free (Ages 
5-17)" REAL, 
        "FRPM Count (Ages 5-17)" REAL, 
        "Percent (%) Eligible FRPM (Ages 
5-17)" REAL, 
        "2013-14 CALPADS Fall 1 Certification
Status" INTEGER, 
        PRIMARY KEY ("CDSCode"), 
        FOREIGN KEY("CDSCode") REFERENCES 
schools ("CDSCode")
)

/*
3 rows from frpm table:
CDSCode Academic Year   County Code     
District Code   School Code     County Name  
District Name   School Name     District Type
School Type     Educational Option Type NSLP 
Provision Status   Charter School (Y/N)    
Charter School Number   Charter Funding Type 
IRC     Low Grade       High Grade      
Enrollment (K-12)       Free Meal Count 
(K-12)  Percent (%) Eligible Free (K-12)     
FRPM Count (K-12)       Percent (%) Eligible 
FRPM (K-12)        Enrollment (Ages 5-17)  
Free Meal Count (Ages 5-17)     Percent (%) 
Eligible Free (Ages 5-17)   FRPM Count (Ages 
5-17)  Percent (%) Eligible FRPM (Ages 5-17) 
2013-14 CALPADS Fall 1 Certification Status
01100170109835  2014-2015       01      10017
0109835 Alameda Alameda County Office of 
Education      FAME Public Charter     County
Office of Education (COE)        K-12 Schools
(Public)   Traditional     None    1       
0728    Directly funded 1       K       12   
1087.0  565.0   0.519779208831647       715.0
0.657773689052438       1070.0  553.0   
0.516822429906542       702.0   
0.65607476635514        1
01100170112607  2014-2015       01      10017
0112607 Alameda Alameda County Office of 
Education      Envision Academy for Arts & 
Technology  County Office of Education (COE) 
High Schools (Public)   Traditional     None 
1       0811    Directly funded 1       9    
12      395.0   186.0   0.470886075949367    
186.0   0.470886075949367       376.0   182.0
0.484042553191489       182.0   
0.484042553191489       1
01100170118489  2014-2015       01      10017
0118489 Alameda Alameda County Office of 
Education      Aspire California College 
Preparatory Academy   County Office of 
Education (COE)        High Schools (Public) 
Traditional     None    1       1049    
Directly funded 1       9       12      244.0
134.0   0.549180327868853       175.0   
0.717213114754098       230.0   128.0   
0.556521739130435       168.0   
0.730434782608696       1
*/


CREATE TABLE satscores (
        cds TEXT NOT NULL, 
        rtype TEXT NOT NULL, 
        sname TEXT, 

::: footer
:::

---

## Calling the LLM with the prompt {.smaller}

`litellm` is a Open Source Python library (and also a Proxy) that can be
used to run inference and compute embeddings with a vast number
of providers (new models added weekly).

Among the provider lists we have:

::: {.incremental}

- Ollama / LMStudio / llama.cpp
- IBM WatsonX.ai
- Anthropic
- OpenAI
- AWS Sagemaker
- OpenRouter
- [and more {{< bi info-circle >}}](https://docs.litellm.ai/docs/providers)
:::

---

### Using litellm

::: {.fragment .fade-in-then-semi-out}

In [20]:
#| echo: true
!uv add litellm --quiet

:::

<!-- [^extras]:   
  `caching` , `extra-proxy` , `mlflow` , `proxy` , `semantic-router`, `utils` -->


::: {.fragment .fade-in}
To run inference, we just call the [`completions`](https://docs.litellm.ai/docs/completion/input) function:

<!-- Enable disk cachesq -->

In [21]:
#| echo: false
# Speed up execution
import litellm 

from litellm.caching.caching import Cache
litellm.cache = Cache(type="disk")
# Remove debug messages
litellm.suppress_debug_info = True

In [22]:
#| echo: true

import litellm

# provider/model (if not local, ensure you have API 🔑 )
model = "watsonx/ibm/granite-3-2-8b-instruct"

response = litellm.completion(
  model=model,
  messages=messages,
)

:::


::: notes

There are some reasonably good LLMs under the coder and instruct in Hugging Face
Some of these can be run locally with some inference server like `Ollama`, 
`llama.cpp` or `LMStudio`, and also use pubic 
ones

:::

---

In [23]:
#| echo: true
#| classes: scrollable
print(response)

ModelResponse(
    id='chatcmpl-f79a3df0-2506-4e7e-aa2b-b65d
21f5d680---5976a0e7cf57f032504389f213b02431--
-e4f953c6-f9cd-4161-bccf-1f1fd819ea7b',
    created=1758486227,
    model='watsonx/ibm/granite-3-2-8b-instruc
t',
    object='chat.completion',
    system_fingerprint=None,
    choices=[
        Choices(
            finish_reason='stop',
            index=0,
            message=Message(
                content='SELECT COUNT(*) FROM
schools\nINNER JOIN frpm ON schools.CDSCode =
frpm.CDSCode\nINNER JOIN satscores ON 
schools.CDSCode = satscores.cds\nWHERE 
satscores."AvgScrMath" > 400\nAND 
schools.Virtual = \'Y\'\nAND schools.Magnet =
0\nAND frpm.CharterSchool(Y/N) = 0\nLIMIT 
10;',
                role='assistant',
                tool_calls=None,
                function_call=None,
                provider_specific_fields=None
            )
        )
    ],
    usage=Usage(
        completion_tokens=102,
        prompt_tokens=3259,
        total_tokens=3361,
        completion_tokens_details=None,
        prompt_tokens_details=None
    ),
    model_id='ibm/granite-3-2-8b-instruct',
    model_version='1.0.0',
    created_at='2025-09-21T20:23:48.449Z',
    system={
        'warnings': [
            {
                'message': "The value of 
'max_completion_tokens' for this model was 
set to value 1024",
                'id': 
'unspecified_max_completion_tokens',
                'additional_properties': {
                    'limit': 0,
                    'new_value': 1024,
                    'parameter': 
'max_completion_tokens',
                    'value': 0
                }
            }
        ]
    }
)

---

## Structured output {.smaller}

::: {.fragment}
LLM outputs are text, SQL for our use case.

It's a good UX to give some explanation to the user about the LLM output. 
We will use `JSON Schema` output to have both output without any string manipulation ✨
:::

::: {.fragment}

In [24]:
#| echo: true
#| code-line-numbers: "1|2-5|9-13|14-17"

from pydantic import BaseModel, Field
class SQLOutput(BaseModel):
    sql: str = Field(description="The SQL query")
    explanation: str = Field(
      description="The reasoning for the query construction")

# Optional 
# litellm.enable_json_schema_validation = True

response = litellm.completion(
  model=model,
  messages=messages,
  response_format=SQLOutput,
)
output = SQLOutput.model_validate_json(
  response.choices[0].message.content)

:::
---

### Attributes of `output` object


In [25]:
#| echo: true
output.sql

"SELECT COUNT(*) FROM schools INNER JOIN satscores ON schools.CDSCode = satscores.cds WHERE schools.Virtual = 'Y' AND satscores.AvgScrMath > 400;"

::: {.fragment}

In [26]:
#| echo: true
output.explanation

"The query selects the count of virtual schools from the 'schools' table that have a 'Virtual' column value of 'Y' and an 'AvgScrMath' (average score in Math) in the 'satscores' table greater than 400. It uses a JOIN to combine data from both tables based on the common 'CDSCode' column."

:::



--- 

### LLM generated SQL vs dataset query

In [27]:
#| echo: true
display(query)
display(output.sql)

"SELECT COUNT(DISTINCT T2.School) FROM satscores AS T1 INNER JOIN schools AS T2 ON T1.cds = T2.CDSCode WHERE T2.Virtual = 'F' AND T1.AvgScrMath > 400"

"SELECT COUNT(*) FROM schools INNER JOIN satscores ON schools.CDSCode = satscores.cds WHERE schools.Virtual = 'Y' AND satscores.AvgScrMath > 400;"

Execution result: 

In [28]:
try:
  with engine.connect() as c:
    response = c.execute(text(output.sql))
    print(response.fetchall())
except Exception as err:
  print(err)

[(0,)]

---

## Improving the Generation with more context

The BIRD mini has some CSV files that help understanding
the tables values and relationships

We're adding this context in a configuration file, that we will examine in the
next section. This is the `evidence` column from the dataset we saw earlier 💡

```{.bash}
 uv run -m sql_with_llms.main ./config-custom.yaml  -c

Provider List: https://docs.litellm.ai/docs/providers


Provider List: https://docs.litellm.ai/docs/providers

Answer: {'sql': "SELECT COUNT(*) FROM satscores INNER JOIN schools ON satscores.cds = schools.CDSCode WHERE satscores.AvgScrMath > 400 AND schools.Virtual = 'F'", 'result': '[(4,)]'}
```



---

# Create a workflow

::: notes

:::

---

# Chaining generation and execution

::: {.fragment .fade-in-then-semi-out}
<img src="./img/langgraph.svg" height="40px" style="float: left; padding-top: .5em; padding-right: .25em"/> 
<p>`LangGraph` allows us to create state machines around LLM calls, using
functions as nodes. The full can be checked out [`src/sql_with_llms/main.py`](https://github.com/D3f0/pycon-sql-llm/blob/main/src/sql_with_llms/main.py)</p>

:::

::: {.incremental}
- Memory (cache)
- Human in the Loop support
- Durable execution
:::

::: notes
We will now put the code we saw into node functions
We will have a `Input`, and `Output` a shared `State` and some global configuration, 
instead of global variables.
:::

---

![](./img/LLMsSQL.svg)


---

## Graph creation

```{.python include="src/sql_with_llms/main.py" snippet="graph" dedent="4" code-line-numbers="false"}
```

::: {.fragment}
Let's take a look at how we define those function and the data types
they receive and return
:::


::: notes
1. In Langraph the node functions must be defined before, but we're presenting this
   before for clarity
:::

---

```{.python}
from sql_with_llms.main import create_graph
g.get_graph().draw_png()
```

![](./img/langraph_graph.png)

---



## Input class

The input and output are these simple dataclasses.

::: {.columns}
::: {.column}
```{.python include="src/sql_with_llms/main.py" snippet="input" code-line-numbers="false"}
```
:::
::: {.column}

```{.python include="src/sql_with_llms/main.py" snippet="output" code-line-numbers="false"}
```

:::
:::

---

## State class

This the state passed between the **inner**.
The first and last one will receive/return the `Input`/`Output` respectively.

```{.python include="src/sql_with_llms/main.py" snippet="state" code-line-numbers="false"}
```

---

## The init node function

This node consumes the input converting the something of shape `Input` into
a `State`. The next node will be prompt generation, so we will be save the
`question`.

```{.python include="src/sql_with_llms/main.py" snippet="init" code-line-numbers="false"}
```

---

## Global configuration class {.smaller}

The nodes beyond the initial one will require to have a bit more information
that won't change (not considered state). We will provide this as a seconds argument 
and bind it with `functools.partial`. 

```{.python include="src/sql_with_llms/main.py" snippet="cfg" code-line-numbers="false"}
```


---

## Prompt generation function node

```{.python include="src/sql_with_llms/main.py" snippet="prompt_gen" code-line-numbers="false"}
```

---


![](./img/LLMsSQL.svg)


---

## LLM calling function node


```{.python include="src/sql_with_llms/main.py" snippet="call_llm" code-line-numbers="false"}
```


## SQL Execution function node

```{.python include="src/sql_with_llms/main.py" snippet="exec_sql" code-line-numbers="false"}
```

---

## Running the code

```{.bash}
git clone https://github.com/D3f0/pycon-sql-llm.git
uv sync
uv run -m sql_with_llms.main 
```

---

# Lessons learned

---

### Lessons learned 

::: {.fragment}
- Tables with a high number of columns can make the context larger
  - RAG at the column level is a common technique to improve efficiency and accuracy.
:::
::: {.fragment}
- SQL manipulation at the abstract syntax tree level (`sqlglot`)
  - Finding dangerous operations (DML)
  - Value substitution 
:::


::: notes
Also we can pass the SQL to SQLAlchemy engine to validate if the syntax is correct
:::

---

### Lessons learned  (cont.)

::: {.fragment}
- Please more context!
  - Adding examples, but not too many, or only the relevant ones 🤔
:::

::: {.fragment}
- Dynamic context

  - date based
  - external resources
  - AI tools
:::


::: footer
For example, the term Q1 can mean, *quarter one*, but it should
    be supplied dynamically
:::
---

### Lessons learned (cont.)

::: {.fragment}
- Function nodes
  - Custom modifications
  - Security (Python on WASM)
:::

---

### Closing words

::: {.fragment}
- [Agentics](https://pypi.org/project/agentics-py/){target="_blank"} is an OSS frameworks for writing agents created at IBM Research
  - Example of text2sql [use case](https://github.com/IBM/Agentics/tree/main/applications/text2sql){target="_blank"}
:::

::: notes

:::
---


## Thank you & Questions{.center}

::: {.columns }
::: {.column width="36%"}
:::
::: {.column width="30%"}
{{< qrcode https://d3f0.github.io/pycon-sql-llm/slides.html#/title-slide qr2 width=400 height=400 colorDark='#0011bb' >}}
:::
::: {.column width="30%"}
:::
:::

::: {style="text-align: center; "}
{{< bi envelope >}} [nahuel.defosse@ibm.com](mailto:nahuel.defosse@ibm.com)
:::